# **Import modules & fetch data from Open Data Melbourne**

### *Method #1 - with limited records (<10000)*

In [ ]:
# import requests
# import pandas as pd
# import os

# # define function to fetch data from website using API

# def fetch_data(base_url, dataset, api_key, num_records = 99, offset = 0):
#     all_records = []
#     max_offset = 9900  # maximum number of requests

#     while True:
#         # maximum limit check
#         if offset > max_offset:
#             break

#         # create API request URL
#         filters = f'{dataset}/records?limit={num_records}&offset={offset}'
#         url = f'{base_url}{filters}&api_key={api_key}'

#         # start request
#         try:
#             result = requests.get(url, timeout=10)
#             result.raise_for_status()
#             records = result.json().get('results')
#         except requests.exceptions.RequestException as e:
#             raise Exception(f"API request failed: {e}")
#         if records is None:
#             break
#         all_records.extend(records)
#         if len(records) < num_records:
#             break

#         # next cycle offset
#         offset += num_records

#     # dataframe all data
#     df = pd.DataFrame(all_records)
#     return df

# API_KEY = os.environ.get('MELBOURNE_API_KEY', input("Please enter your API key: "))
# BASE_URL = 'https://data.melbourne.vic.gov.au/api/explore/v2.1/catalog/datasets/'

In [ ]:
# # get data
# df_id = 'customer-service-requests-with-resolution-time'

# df = fetch_data(BASE_URL, df_id, API_KEY)

# print(f'Number of records: {len(df)}')

### *Method #2 - with unlimited data records (credit to Te's API tutorial)*

In [ ]:
from google.colab import drive
import requests
import os
from io import StringIO

import pandas as pd
import calendar

In [ ]:
drive.mount('/content/gdrive')

with open('/content/gdrive/My Drive/Colab Notebooks/apiKey.txt', 'r') as file:
    api_key = file.read().strip()

api_key = os.getenv(api_key)

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


# **Dataset:** Melbourne Visitor Contact Stats

## Load dataset

In [ ]:
# export endpoint

# https://data.melbourne.vic.gov.au/explore/dataset/melbourne-visitor-contact-stats/table/
dataset1_id = 'melbourne-visitor-contact-stats'

base_URL = 'https://data.melbourne.vic.gov.au/api/explore/v2.1/catalog/datasets/'
apikey = api_key
dataset1_id = dataset1_id
format = 'csv'

url = f'{base_URL}{dataset1_id}/exports/{format}'
params = {
    'select': '*',
    'limit': -1,  # all records
    'lang': 'en',
    'timezone': 'UTC',
    'api_key': apikey
}

# GET request
response = requests.get(url, params = params)

if response.status_code == 200:
    # StringIO to read the CSV data
    url_content = response.content.decode('utf-8')
    MelVisitor_stat = pd.read_csv(StringIO(url_content), delimiter = ';')
    print(MelVisitor_stat.sample(3, random_state = 999))    # test
else:
    print(f'Request failed with status code {response.status_code}')

           date  number_of_contacts                   source  year     month
258  2017-01-11               21253  Melbourne Visitor Booth  2017  November
0    2018-01-12               21661  Melbourne Visitor Booth  2018  December
247  2018-01-05                   0      Cruise Ship Program  2018       May


In [ ]:
# check length in dataset 1 (dataset1_id)

print(f'The dataset contains {len(MelVisitor_stat)} records.')

The dataset contains 476 records.


## Data cleansing

In [ ]:
print(f'Number of missing values in each column:\n\n{MelVisitor_stat.isna().sum()}')

Number of missing values in each column:

date                  0
number_of_contacts    0
source                0
year                  0
month                 0
dtype: int64


In [ ]:
print(f"Number of unique sources: {MelVisitor_stat['source'].nunique()}\n")
print(f"{MelVisitor_stat['source'].unique()}")

Number of unique sources: 12

['Melbourne Visitor Booth' 'Melbourne Greeter Service'
 'Melbourne Visitor Shuttle' 'Melbourne Visitor Centre'
 'City Ambassador Program' 'Inbox email enquiries' 'Inbox email Inquiries'
 'Cruise Ship Program' "Cooks' Cottage" 'Fitzroy Gardens Visitor Centre'
 'Melbourne Visitor Hub at Town Hall' 'Queen Victoria Market Visitor Hub']


Based on the analysis of unique values from the 'source' column, and considering the termination and opening of certain visitor service centers as follows, 'Inbox email enquiries' and 'Inbox email Inquiries' have been identified as duplicates representing the same source. To ensure consistency and avoid misleading results, these entries have been consolidated under a single label. Furthermore, while the Melbourne Visitor Hub at Town Hall could be regarded as a continuation or relocation of the Melbourne Visitor Centre, they will be analyzed as separate entities in this analysis due to their distinct locations and potential differences in visitor interactions.

Significant dates to note include:
*   The Melbourne Visitor Shuttle ceased operations in September 2017
*   The Melbourne Visitor Centre was closed on Sunday, 19th of August, 2018
*   The Melbourne Visitor Hub at Town Hall opened on Monday, 20th of August, 2018
*   The Visitor Hub at Queen Victoria Market opened in late November 2018

In [ ]:
# replacing 'Inbox email Inquiries' with 'Inbox email enquires' to ensure consistency
MelVisitor_stat['source'] = MelVisitor_stat['source'].replace(
    {'Inbox email Inquiries': 'Inbox email enquiries'})

print(f"Number of unique sources: {MelVisitor_stat['source'].nunique()}\n\n{MelVisitor_stat['source'].unique()}")

Number of unique sources: 11

['Melbourne Visitor Booth' 'Melbourne Greeter Service'
 'Melbourne Visitor Shuttle' 'Melbourne Visitor Centre'
 'City Ambassador Program' 'Inbox email enquiries' 'Cruise Ship Program'
 "Cooks' Cottage" 'Fitzroy Gardens Visitor Centre'
 'Melbourne Visitor Hub at Town Hall' 'Queen Victoria Market Visitor Hub']


In [ ]:
MelVisitor_stat

,date,number_of_contacts,source,year,month
0,2018-01-12,21661,Melbourne Visitor Booth,2018,December
1,2018-01-12,171,Melbourne Greeter Service,2018,December
2,2015-01-09,9028,Melbourne Visitor Shuttle,2015,September
3,2015-01-07,68793,Melbourne Visitor Centre,2015,July
4,2014-01-11,25151,City Ambassador Program,2014,November
...,...,...,...,...,...
471,2016-01-01,25546,Fitzroy Gardens Visitor Centre,2016,January
472,2018-01-02,6402,Cooks' Cottage,2018,February
473,2017-01-10,0,Melbourne Visitor Shuttle,2017,October
474,2017-01-07,50134,Melbourne Visitor Centre,2017,July


## Aggregation

The aim is to calculate the total number of contacts at Visitor Service Centres, aggregated by year and month, which will help us understand potential trends in visitor engagement over the years, providing insight into how engagement levels have changed over time.

In [ ]:
print(MelVisitor_stat.dtypes)

date                  object
number_of_contacts     int64
source                object
year                   int64
month                 object
dtype: object


In [ ]:
# convert column 'date' to date format
MelVisitor_stat['date'] = pd.to_datetime(MelVisitor_stat['date'])

# convert column 'month' from strings to numbers
month_num = {month: index for index, month in enumerate(calendar.month_name) if month}

MelVisitor_stat['month'] = MelVisitor_stat['month'].map(month_num)

In [ ]:
print(MelVisitor_stat.dtypes)

date                  datetime64[ns]
number_of_contacts             int64
source                        object
year                           int64
month                          int64
dtype: object


In [ ]:
# check the first few rows to confirm the conversion
print(MelVisitor_stat['date'].head())

0   2018-01-12
1   2018-01-12
2   2015-01-09
3   2015-01-07
4   2014-01-11
Name: date, dtype: datetime64[ns]


In [ ]:
sum_contacts = MelVisitor_stat.groupby(['year', 'month'])['number_of_contacts'].sum().reset_index()
sum_contacts = sum_contacts.sort_values(by = ['year', 'month'], ascending = [True, True])

In [ ]:
print(sum_contacts.head(10))

   year  month  number_of_contacts
0  2014      7              162589
1  2014      8              137713
2  2014      9              136758
3  2014     10              150532
4  2014     11              164042
5  2014     12              193392
6  2015      1              206044
7  2015      2              194113
8  2015      3              205147
9  2015      4              166348


In [ ]:
print(type(sum_contacts))

<class 'pandas.core.frame.DataFrame'>


## Observations

In [ ]:
# create 'date' column formatted as a string "YYYY-MM"
sum_contacts['date'] = sum_contacts['year'].astype(str) + '-' + sum_contacts['month'].astype(str).str.zfill(2)

sum_contacts.head(2)

,year,month,number_of_contacts,date
0,2014,7,162589,2014-07
1,2014,8,137713,2014-08


In [ ]:
# !pip install plotly

# import matplotlib.pyplot as plt
# import seaborn as sns
import plotly.express as px

In [ ]:
# create interactive line plot
fig_line = px.line(sum_contacts, x = 'date', y = 'number_of_contacts',
                               title = 'Total Number of Contacts at Visitor Service Centres Over Time',
                               labels = {'date': 'Date', 'number_of_contacts': 'Number of Contacts'})

# improve layout
fig_line.update_layout(
    title = {
        'text': 'Total Number of Contacts at Visitor Service Centres Over Time',
        'y': 0.9,
        'x': 0.5,
        'xanchor': 'center',
        'yanchor': 'top'
        },
    hovermode = 'closest',  # show tooltip for the closest point
    xaxis = dict(tickmode = 'auto', nticks = 20, dtick = "M3", tickformat = "%Y-%m")
)

# show the plot
fig_line.show()

This line chart reveals notable trends in the number of contacts at Visitor Service Centres over time. It is particularly striking that contact volumes peak from December to March each year, while the period from May to September typically experiences the lowest number of visitor contacts. These observations could provide valuable insights for customer service centers, as they may correlate with fluctuations in the volume or nature of customer service requests. To explore this further, additional datasets will be analyzed to examine the relationship between periods of high visitor engagement and increased customer service requests. The integration of these data will allow for a more comprehensive analysis of trends and may reveal actionable correlations between visitor contact stats and customer service requests.

# **Dataset:** Customer service requests, with resolution time

## Load dataset

In [ ]:
# export endpoint

# https://data.melbourne.vic.gov.au/explore/dataset/customer-service-requests-with-resolution-time/information/
dataset2_id = 'customer-service-requests-with-resolution-time'

base_URL = 'https://data.melbourne.vic.gov.au/api/explore/v2.1/catalog/datasets/'
apikey = api_key
dataset2_id = dataset2_id
format = 'csv'

url = f'{base_URL}{dataset2_id}/exports/{format}'
params = {
    'select': '*',
    'limit': -1,  # all records
    'lang': 'en',
    'timezone': 'UTC',
    'api_key': apikey
}

# GET request
response = requests.get(url, params=params)

if response.status_code == 200:
    # StringIO to read the CSV data
    url_content = response.content.decode('utf-8')
    CS_TTR = pd.read_csv(StringIO(url_content), delimiter = ';')
    print(CS_TTR.sample(2, random_state = 999))    # test
else:
    print(f'Request failed with status code {response.status_code}')

      request_status date_received date_completed           suburb  \
22471         CLOSED    2015-10-19     2015-10-28              NaN   
4040          CLOSED    2016-04-07     2016-04-14  North Melbourne   

                                category              service_desc  \
22471  Waste, Street Cleaning and Litter  Street Cleaning services   
4040                            Graffiti          Graffiti Removal   

       days_to_complete  
22471                 9  
4040                  7  


In [ ]:
# check length in dataset 2 (dataset2_id)

print(f'The dataset contains {len(CS_TTR)} records.')

The dataset contains 46479 records.


In [ ]:
print(f'Number of missing values in each column:\n\n{CS_TTR.isna().sum()}')

Number of missing values in each column:

request_status          0
date_received           0
date_completed        848
suburb              24894
category                0
service_desc            0
days_to_complete        0
dtype: int64


## Data cleansing

In [ ]:
print(CS_TTR.dtypes)

request_status      object
date_received       object
date_completed      object
suburb              object
category            object
service_desc        object
days_to_complete     int64
dtype: object


*Upon reviewing the dataset and its schema, it became clear that the date columns were not in the correct datetime format, a necessary conversion for any time-based calculations in later analysis. The 'suburb' and 'category' columns, which contain categorical data, could potentially be encoded for more efficient processing in subsequent steps. However, I will refrain from encoding at this stage, as these columns may be merged with another dataset to create a more comprehensive analysis later on.*

In [ ]:
# convert columns 'date_received' and 'date_completed' to datetime object

CS_TTR['date_received'] = pd.to_datetime(CS_TTR['date_received'])
CS_TTR['date_completed'] = pd.to_datetime(CS_TTR['date_completed'], errors = 'coerce')

In [ ]:
print(CS_TTR.dtypes)

request_status              object
date_received       datetime64[ns]
date_completed      datetime64[ns]
suburb                      object
category                    object
service_desc                object
days_to_complete             int64
dtype: object


*I have determined that it is reasonable to presume that some requests may not have been completed at the time the data was collected. Consequently, it is appropriate to maintain these records as 'NaN' instead of imputing artificial completion dates. These rows will be preserved within the 'date_completed' column, as they are indicative of actual service requests, irrespective of their completion status. However, they may need to be excluded from future analyses involving resolution times.*

*Previously, I entertained the idea of performing a geographical analysis, but I have since decided against it due to the current lack of sufficient geographical data. Adhering to my initial objectives, which focus on the volume of requests, categories, and resolution times, I deemed the 'suburb' information non-essential. Moreover, given that more than half of the dataset's records lack 'suburb' data, removing these rows could significantly diminish the dataset's size and introduce potential biases.*

*Given these considerations, reclassifying the missing 'suburb' data as 'Unknown' or 'Not Applicable' appears to be the most prudent course of action. This will preserve the integrity of the dataset and maintain the option for geographical analysis should it become pertinent in the future.*

In [ ]:
# replace the 'NaN' values in 'suburb' column with 'Unknown'

CS_TTR['suburb'].fillna('Unknown', inplace = True)

# **Dataset:** Customer Service Requests (with digital channel information)

## Load dataset

In [ ]:
# export endpoint

# https://data.melbourne.vic.gov.au/explore/dataset/customer-service-requests-with-digital-channel-information/table/
dataset3_id = 'customer-service-requests-with-digital-channel-information'

base_URL = 'https://data.melbourne.vic.gov.au/api/explore/v2.1/catalog/datasets/'
apikey = api_key
dataset3_id = dataset3_id
format = 'csv'

url = f'{base_URL}{dataset3_id}/exports/{format}'
params = {
    'select': '*',
    'limit': -1,  # all records
    'lang': 'en',
    'timezone': 'UTC',
    'api_key': apikey
}

# GET request
response = requests.get(url, params = params)

if response.status_code == 200:
    # StringIO to read the CSV data
    url_content = response.content.decode('utf-8')
    CS_digital = pd.read_csv(StringIO(url_content), delimiter = ';')
    print(CS_digital.sample(2, random_state = 999))    # test
else:
    print(f'Request failed with status code {response.status_code}')

                   service_type                date_opened  \
37845        Syringe Management  2018-01-09T13:20:00+00:00   
381    Tree Maintenance Request  2018-02-10T11:14:00+00:00   

                     date_closed  status case_origin     suburb  
37845  2018-01-09T14:00:00+00:00  Closed         Web  Southbank  
381    2018-02-21T13:36:00+00:00  Closed         Web  Melbourne  


In [ ]:
# check length in dataset 3 (dataset3_id)

print(f'The dataset contains {len(CS_digital)} records.')

The dataset contains 42221 records.


In [ ]:
print(CS_digital.isna().sum())

service_type        0
date_opened         0
date_closed      9491
status              0
case_origin         0
suburb          22618
dtype: int64


In [ ]:
print(CS_digital.dtypes)

service_type    object
date_opened     object
date_closed     object
status          object
case_origin     object
suburb          object
dtype: object


In [ ]:
CS_digital['date_opened'] = pd.to_datetime(CS_digital['date_opened'])
CS_digital['date_closed'] = pd.to_datetime(CS_digital['date_closed'])

In [ ]:
print(CS_digital.dtypes)

service_type                 object
date_opened     datetime64[ns, UTC]
date_closed     datetime64[ns, UTC]
status                       object
case_origin                  object
suburb                       object
dtype: object


In [ ]:
# replace the 'NaN' values in 'suburb' column with 'Unknown'

CS_digital['suburb'].fillna('Unknown', inplace = True)

In [ ]:
CS_digital_copy = CS_digital.copy()

In [ ]:
# to convert the datetime object contain date values only

CS_digital_copy['date_opened'] = pd.to_datetime(CS_digital['date_opened'].dt.date)
CS_digital_copy['date_closed'] = pd.to_datetime(CS_digital['date_closed'].dt.date)

In [ ]:
print(CS_digital_copy.head(2))

                    service_type date_opened date_closed status case_origin  \
0  Infringement Review - Parking  2018-05-31         NaT    New       Phone   
1  Infringement Review - Parking  2018-04-24         NaT    New        Mail   

    suburb  
0  Unknown  
1  Unknown  


In [ ]:
print(CS_digital_copy.dtypes)

service_type            object
date_opened     datetime64[ns]
date_closed     datetime64[ns]
status                  object
case_origin             object
suburb                  object
dtype: object


# **Combining datasets**

In [ ]:
# align and rename columns in these two dataset

CS_digital_copy = CS_digital_copy.rename(
    columns = {
        'date_opened': 'date_received',
        'date_closed': 'date_completed'
    }
)

CS_TTR = CS_TTR.rename(
    columns = {'service_desc': 'service_type','request_status': 'status'}
)

In [ ]:
print(CS_digital_copy.head(2))

                    service_type date_received date_completed status  \
0  Infringement Review - Parking    2018-05-31            NaT    New   
1  Infringement Review - Parking    2018-04-24            NaT    New   

  case_origin   suburb  
0       Phone  Unknown  
1        Mail  Unknown  


In [ ]:
print(CS_TTR.head(2))

   status date_received date_completed          suburb  \
0  CLOSED    2016-05-05     2016-05-12         Carlton   
1  CLOSED    2014-11-20     2014-11-21  East Melbourne   

                            category           service_type  days_to_complete  
0                           Graffiti       Graffiti Removal                 7  
1  Waste, Street Cleaning and Litter  Missed Bin Collection                 1  


In [ ]:
df = pd.concat([CS_TTR, CS_digital_copy], ignore_index = True)

In [ ]:
print(df.head(2))

   status date_received date_completed          suburb  \
0  CLOSED    2016-05-05     2016-05-12         Carlton   
1  CLOSED    2014-11-20     2014-11-21  East Melbourne   

                            category           service_type  days_to_complete  \
0                           Graffiti       Graffiti Removal               7.0   
1  Waste, Street Cleaning and Litter  Missed Bin Collection               1.0   

  case_origin  
0         NaN  
1         NaN  


In [ ]:
# check length in the merged dataset

print(f'This combined dataset contains {len(df)} records.')

This combined dataset contains 88700 records.


In [ ]:
print(df.dtypes)

status                      object
date_received       datetime64[ns]
date_completed      datetime64[ns]
suburb                      object
category                    object
service_type                object
days_to_complete           float64
case_origin                 object
dtype: object


In [ ]:
print(f'Number of missing values in each column:\n\n{df.isna().sum()}')

Number of missing values in each column:

status                  0
date_received           0
date_completed      10339
suburb                  0
category            42221
service_type            0
days_to_complete    42221
case_origin         46479
dtype: int64


In [ ]:
# filling 'NaN' in 'case_origin' column with 'Unknown'
df['case_origin'].fillna('Unknown', inplace = True)

In [ ]:
print(f'Number of missing values in each column:\n\n{df.isna().sum()}')

Number of missing values in each column:

status                  0
date_received           0
date_completed      10339
suburb                  0
category            42221
service_type            0
days_to_complete    42221
case_origin             0
dtype: int64


In [ ]:
print(df[df['category'].isna()]['service_type'].unique())

['Infringement Review - Parking' 'Graffiti Removal Request'
 'Illegally Dumped Rubbish' 'Request for Parking Officer' 'Rough Sleeping'
 'Infringement Review' 'Tree Maintenance Request' 'Abandoned Vehicles'
 'Syringe Management' 'Infringement Review - Local Law' 'Ratepayer waste'
 'General Enquiry' 'Other' 'Infringement Review - Animal Management']


In [ ]:
print(f"Number of categories: {df['category'].nunique()}\n\n{df['category'].unique()}")

Number of categories: 6

['Graffiti' 'Waste, Street Cleaning and Litter' 'Parks and Trees'
 'Roads and Traffic' 'Asset maintenance' 'Parking' nan]


In [ ]:
print(f"Unique values in 'Waste, Street Cleaning and Litter' category: \n\n{df[df['category'] == 'Waste, Street Cleaning and Litter']['service_type'].unique()}")

Unique values in 'Waste, Street Cleaning and Litter' category: 

['Missed Bin Collection' 'Street Cleaning services' 'Damaged Bins'
 'Public Litter Bin' 'Missing Bin' 'Waste collection services'
 'Dumped Rubbish' 'Syringe pick-up services' 'Organic Waste'
 'Waste Compactor' 'Waterways']


In [ ]:
print(f"Unique values in 'Graffiti' category: \n\n{df[df['category'] == 'Graffiti']['service_type'].unique()}")

Unique values in 'Graffiti' category: 

['Graffiti Removal']


In [ ]:
print(f"Unique values in 'Roads and Traffic' category: \n\n{df[df['category'] == 'Roads and Traffic']['service_type'].unique()}")

Unique values in 'Roads and Traffic' category: 

['Road and Footpath Maintenance' 'Traffic Management' 'Drain Maintenance']


In [ ]:
print(f"Unique values in 'Parking' category:\n\n{df[df['category'] == 'Parking']['service_type'].unique()}")

Unique values in 'Parking' category:

['Parking Compliance Services' 'Parking Meter Service']


In [ ]:
print(f"Unique values in 'Parks and Trees' category:\n\n{df[df['category'] == 'Parks and Trees']['service_type'].unique()}")

Unique values in 'Parks and Trees' category:

['Park Cleaning' 'Tree Maintenance Services'
 'Condition of Assets in Parks' 'Lawns and Irrigation'
 'Sport and Playground Facilities']


In [ ]:
print(f"Unique values in 'Asset maintenance' category:\n\n{df[df['category'] == 'Asset maintenance']['service_type'].unique()}")

Unique values in 'Asset maintenance' category:

['Street Maintenance' 'Street Lighting Maintenance' 'Public Toilets'
 'Bridge Maintenance' 'Bike pod services']


After examination, each service type will be grouped to:

**[Graffiti]**
*   Graffiti Removal
*   Graffiti Removal Request

**[Waste, Street Cleaning and Litter]**
*   Missed Bin Collection
*   Street Cleaning services
*   Damaged Bins
*   Public Litter Bin
*   Missing Bin
*   Waste collection services
*   Dumped Rubbish
*   Syringe pick-up services
*   Organic Waste
*   Waste Compactor
*   Waterways
*   Syringe Management
*   Illegally Dumped Rubbish
*   Ratepayer waste

**[Parks and Trees]**
*   Park Cleaning
*   Tree Maintenance Services
*   Condition of Assets in Parks
*   Lawns and Irrigation
*   Sport and Playground Facilities
*   Tree Maintenance Request
*   Rough Sleeping

**[Roads and Traffic]**
*   Road and Footpath Maintenance
*   Traffic Management
*   Drain Maintenance

**[Asset maintenance]**
*   Street Maintenance
*   Street Lighting Maintenance
*   Public Toilets
*   Bridge Maintenance
*   Bike pod services

**[Parking]**
*   Parking Compliance Services
*   Parking Meter Service
*   Request for Parking Officer
*   Abandoned Vehicles

**[General]**
*   Infringement Review
*   Infringement Review - Parking
*   Infringement Review - Animal Management
*   Infringement Review - Local Law
*   General Enquiry
*   Other

In [ ]:
# define categorization mapping

categorization = {
    'Infringement Review': 'General',
    'Infringement Review - Parking': 'General',
    'Infringement Review - Animal Management': 'General',
    'Infringement Review - Local Law': 'General',
    'General Enquiry': 'General',
    'Other': 'General',
    'Graffiti Removal Request': 'Graffiti',
    'Syringe Management': 'Waste, Street Cleaning and Litter',
    'Illegally Dumped Rubbish': 'Waste, Street Cleaning and Litter',
    'Ratepayer waste': 'Waste, Street Cleaning and Litter',
    'Request for Parking Officer': 'Parking',
    'Abandoned Vehicles': 'Parking',
    'Tree Maintenance Request': 'Parks and Trees',
    'Rough Sleeping': 'Parks and Trees'
    }

# missing_category = df['category'].isnull()

# apply the categorization
df.loc[df['category'].isnull(), 'category'] = df[df['category'].isnull()]['service_type'].map(categorization)

In [ ]:
print(df.isna().sum())

status                  0
date_received           0
date_completed      10339
suburb                  0
category                0
service_type            0
days_to_complete    42221
case_origin             0
dtype: int64


In [ ]:
# address the duplication in service types

replacements = {
    'Graffiti Removal Request': 'Graffiti Removal',
    'Syringe pick-up services': 'Syringe Management',
    'Tree Maintenance Request': 'Tree Maintenance Services',
    'Infringement Review': 'Infringement Review - Undefined',
    'Dumped Rubbish': 'Illegally Dumped Rubbish',}

df['service_type'] = df['service_type'].replace(replacements)

In [ ]:
print(f"All unique service types: {df['service_type'].nunique()}\n\n{df['service_type'].unique()}")

All unique service types: 37

['Graffiti Removal' 'Missed Bin Collection' 'Street Cleaning services'
 'Damaged Bins' 'Public Litter Bin' 'Park Cleaning'
 'Tree Maintenance Services' 'Condition of Assets in Parks'
 'Road and Footpath Maintenance' 'Missing Bin' 'Lawns and Irrigation'
 'Waste collection services' 'Illegally Dumped Rubbish'
 'Syringe Management' 'Street Maintenance' 'Parking Compliance Services'
 'Street Lighting Maintenance' 'Public Toilets' 'Traffic Management'
 'Sport and Playground Facilities' 'Drain Maintenance' 'Organic Waste'
 'Bridge Maintenance' 'Parking Meter Service' 'Waste Compactor'
 'Bike pod services' 'Waterways' 'Infringement Review - Parking'
 'Request for Parking Officer' 'Rough Sleeping'
 'Infringement Review - Undefined' 'Abandoned Vehicles'
 'Infringement Review - Local Law' 'Ratepayer waste' 'General Enquiry'
 'Other' 'Infringement Review - Animal Management']


In [ ]:
print(f"All unique status: {df['status'].nunique()}\n\n{df['status'].unique()}")

All unique status: 15

['CLOSED' 'WIP' 'NEW' 'REASSIGNED' 'ACTIONED' 'New' 'Closed'
 'Assigned for Action' 'In Progress' 'Closed as Duplicate'
 'Awaiting Customer Information' 'Escalated' 'Actioned' 'TL Action'
 'Pending Closure']


In [ ]:
# # convert 'status' to lower case to standardize it
# df['status'] = df['status'].str.lower()

# capitalize only the first letter of each word in 'status'
df['status'] = df['status'].str.title()

# address the duplication in 'status' column
status_replacement = {
    'Wip': 'In Progress'
}

df['status'] = df['status'].replace(status_replacement)

In [ ]:
print(f"All unique status: {df['status'].nunique()}\n\n{df['status'].unique()}")

All unique status: 11

['Closed' 'In Progress' 'New' 'Reassigned' 'Actioned'
 'Assigned For Action' 'Closed As Duplicate'
 'Awaiting Customer Information' 'Escalated' 'Tl Action' 'Pending Closure']


In [ ]:
print(f"All unique suburb: {df['suburb'].nunique()}\n\n{df['suburb'].unique()}")

All unique suburb: 45

['Carlton' 'East Melbourne' 'Melbourne' 'Southbank' 'North Melbourne'
 'Parkville' 'Docklands' 'West Melbourne' 'South Yarra' 'Kensington'
 'Carlton North' 'Port Melbourne' 'Unknown' 'South Wharf' 'Flemington'
 'carlton' 'Melbourne CBD' 'Kensignton' 'Princes Hill' 'Richmond'
 'Southgate Promenade' 'West Melb' 'kensington' 'north melbourne'
 'west melb' 'North melbourne' 'Deer Park' 'melbourne' 'Batman Ave'
 'Fitzroy' 'MELBOURNE' 'North Adelaide' 'South Melbourne' 'Melb CBD'
 'Jolimont' 'Brunswick' 'Elizabeth' 'West melbourne' 'Royal Park'
 'west melbourne' 'Prahran' 'Nth melbourne' 'CARLTON' 'EAST MELBOURNE'
 'Reservoir']


In [ ]:
# capitalize only the first letter of each word in 'suburb' to standardize it
df['suburb'] = df['suburb'].str.title()

# address the duplication in 'suburb' column
corrections = {
    'Nth Melbourne': 'North Melbourne',
    'Kensignton': 'Kensington',
    'Melbourne Cbd': 'Melbourne CBD',
    'Melb Cbd': 'Melbourne CBD',
    'West Melb': 'West Melbourne'
}

df['suburb'] = df['suburb'].replace(corrections)

In [ ]:
print(f"All unique suburb: {df['suburb'].nunique()}\n\n{df['suburb'].unique()}")

All unique suburb: 30

['Carlton' 'East Melbourne' 'Melbourne' 'Southbank' 'North Melbourne'
 'Parkville' 'Docklands' 'West Melbourne' 'South Yarra' 'Kensington'
 'Carlton North' 'Port Melbourne' 'Unknown' 'South Wharf' 'Flemington'
 'Melbourne CBD' 'Princes Hill' 'Richmond' 'Southgate Promenade'
 'Deer Park' 'Batman Ave' 'Fitzroy' 'North Adelaide' 'South Melbourne'
 'Jolimont' 'Brunswick' 'Elizabeth' 'Royal Park' 'Prahran' 'Reservoir']


In [ ]:
df.head(3)

,status,date_received,date_completed,suburb,category,service_type,days_to_complete,case_origin
0,Closed,2016-05-05,2016-05-12,Carlton,Graffiti,Graffiti Removal,7.0,Unknown
1,Closed,2014-11-20,2014-11-21,East Melbourne,"Waste, Street Cleaning and Litter",Missed Bin Collection,1.0,Unknown
2,Closed,2014-11-27,2014-12-01,Carlton,"Waste, Street Cleaning and Litter",Missed Bin Collection,4.0,Unknown


In [ ]:
print(df.isnull().sum())

status                  0
date_received           0
date_completed      10339
suburb                  0
category                0
service_type            0
days_to_complete    42221
case_origin             0
dtype: int64


In [ ]:
# address the missing values in 'days_to_complete' column
# calculate the difference in days

df['days_to_complete'] = (df['date_completed'] - df['date_received']).dt.days

In [ ]:
print(df.isnull().sum())

status                  0
date_received           0
date_completed      10339
suburb                  0
category                0
service_type            0
days_to_complete    10339
case_origin             0
dtype: int64


In [ ]:
print(df.dtypes)

status                      object
date_received       datetime64[ns]
date_completed      datetime64[ns]
suburb                      object
category                    object
service_type                object
days_to_complete           float64
case_origin                 object
dtype: object


In [ ]:
# convert 'days_to_complete' to a nullable integer type

df['days_to_complete'] = df['days_to_complete'].astype('Int64')

In [ ]:
print(df.dtypes)

status                      object
date_received       datetime64[ns]
date_completed      datetime64[ns]
suburb                      object
category                    object
service_type                object
days_to_complete             Int64
case_origin                 object
dtype: object


In [ ]:
df = df.sort_values(by = 'date_received', ascending = True).reset_index(drop = True)

In [ ]:
df

,status,date_received,date_completed,suburb,category,service_type,days_to_complete,case_origin
0,Closed,2014-11-10,2015-09-20,Unknown,Parking,Parking Meter Service,314,Unknown
1,Closed,2014-11-10,2014-11-10,Unknown,"Waste, Street Cleaning and Litter",Street Cleaning services,0,Unknown
2,Closed,2014-11-10,2014-11-12,Parkville,"Waste, Street Cleaning and Litter",Missed Bin Collection,2,Unknown
3,Closed,2014-11-10,2014-11-11,Kensington,"Waste, Street Cleaning and Litter",Illegally Dumped Rubbish,1,Unknown
4,Closed,2014-11-10,2014-11-12,Unknown,"Waste, Street Cleaning and Litter",Illegally Dumped Rubbish,2,Unknown
...,...,...,...,...,...,...,...,...
88695,New,2018-09-30,NaT,Unknown,General,Infringement Review - Parking,<NA>,Web
88696,New,2018-09-30,NaT,Unknown,General,Infringement Review - Parking,<NA>,Web
88697,Closed,2018-09-30,2018-09-30,Carlton,Parking,Request for Parking Officer,0,Web
88698,New,2018-09-30,NaT,Unknown,General,Infringement Review - Parking,<NA>,Web


## Aggregation

In [ ]:
# extract year and month from 'date_received'
df['rec_year'] = df['date_received'].dt.year
df['rec_month'] = df['date_received'].dt.month

In [ ]:
df.head(5)

,status,date_received,date_completed,suburb,category,service_type,days_to_complete,case_origin,rec_year,rec_month
0,Closed,2014-11-10,2015-09-20,Unknown,Parking,Parking Meter Service,314,Unknown,2014,11
1,Closed,2014-11-10,2014-11-10,Unknown,"Waste, Street Cleaning and Litter",Street Cleaning services,0,Unknown,2014,11
2,Closed,2014-11-10,2014-11-12,Parkville,"Waste, Street Cleaning and Litter",Missed Bin Collection,2,Unknown,2014,11
3,Closed,2014-11-10,2014-11-11,Kensington,"Waste, Street Cleaning and Litter",Illegally Dumped Rubbish,1,Unknown,2014,11
4,Closed,2014-11-10,2014-11-12,Unknown,"Waste, Street Cleaning and Litter",Illegally Dumped Rubbish,2,Unknown,2014,11


In [ ]:
df[df['suburb'] != 'Unknown']

,status,date_received,date_completed,suburb,category,service_type,days_to_complete,case_origin,rec_year,rec_month
2,Closed,2014-11-10,2014-11-12,Parkville,"Waste, Street Cleaning and Litter",Missed Bin Collection,2,Unknown,2014,11
3,Closed,2014-11-10,2014-11-11,Kensington,"Waste, Street Cleaning and Litter",Illegally Dumped Rubbish,1,Unknown,2014,11
10,Closed,2014-11-10,2014-11-11,East Melbourne,Parks and Trees,Condition of Assets in Parks,1,Unknown,2014,11
14,Closed,2014-11-10,2014-11-13,Kensington,"Waste, Street Cleaning and Litter",Missing Bin,3,Unknown,2014,11
17,Closed,2014-11-10,2014-11-21,Melbourne,Parks and Trees,Tree Maintenance Services,11,Unknown,2014,11
...,...,...,...,...,...,...,...,...,...,...
88686,Closed,2018-09-30,2018-10-04,Parkville,"Waste, Street Cleaning and Litter",Illegally Dumped Rubbish,4,Web,2018,9
88687,Closed,2018-09-30,2018-10-09,Melbourne,Parking,Abandoned Vehicles,9,Web,2018,9
88688,Closed,2018-09-30,2018-09-30,Southbank,Parking,Request for Parking Officer,0,Web,2018,9
88693,Closed,2018-09-30,2018-09-30,Melbourne,Parks and Trees,Rough Sleeping,0,Phone,2018,9


In [ ]:
# group by 'category', 'year' and 'month', then count the number of cases
mth_sum_category = df.groupby(['category', 'rec_year', 'rec_month']).size().reset_index(name = 'total_cases')

# rename columns for better readability
mth_sum_category.rename(columns = {'rec_year': 'year', 'rec_month': 'month'}, inplace = True)

# sort the results
mth_sum_category.sort_values(by = ['year', 'month', 'category'], inplace = True)

In [ ]:
# create 'date' column formatted as a string "YYYY-MM"
mth_sum_category['date'] = mth_sum_category['year'].astype(str) + '-' + mth_sum_category['month'].astype(str).str.zfill(2)

mth_sum_category = mth_sum_category[['category', 'year', 'month', 'date', 'total_cases']]

In [ ]:
mth_sum_category.reset_index(drop = True)

,category,year,month,date,total_cases
0,Asset maintenance,2014,11,2014-11,56
1,Graffiti,2014,11,2014-11,120
2,Parking,2014,11,2014-11,57
3,Parks and Trees,2014,11,2014-11,234
4,Roads and Traffic,2014,11,2014-11,124
...,...,...,...,...,...
190,General,2018,9,2018-09,2248
191,Graffiti,2018,9,2018-09,497
192,Parking,2018,9,2018-09,1046
193,Parks and Trees,2018,9,2018-09,327


In [ ]:
# find the sum of all the cases across each category when the month is 11
november_cases_sum = mth_sum_category[mth_sum_category['month'] == 11]['total_cases'].sum()
november_cases_sum

3520

# **Observations**

### Heat map

In [ ]:
# use pivot_table to handle potential duplicate entries by aggregating them
heatmap_data = mth_sum_category.pivot_table(index = 'category',
                                            columns = 'month',
                                            values = 'total_cases',
                                            aggfunc = 'sum').fillna(0)

# create interactive heatmap
fig_heatmap = px.imshow(heatmap_data,
                        labels = dict(x = "Month", y = "Category", color = "Total Cases"),
                        x = [f'mth {i}' for i in range(1, 13)],    # months are numbered from 1 through 12
                        y = heatmap_data.index,
                        color_continuous_scale = "deep")

fig_heatmap.update_layout(title = {'text': 'Density of Cases Across Months and Categories',
                                   'y': 0.95, 'x': 0.5,
                                   'xanchor': 'center',
                                   'yanchor': 'top'},
                          coloraxis_colorbar = dict(title = dict(side = 'right', text = 'Total Cases'),
                                                    yanchor = 'middle',
                                                    ticks = 'outside',
                                                    ticksuffix = ' '))

fig_heatmap.show()

*The heatmap of case density across months and categories shows distinct patterns that may suggest seasonal trends and category-specific demand for services. For example, certain categories like 'Waste, Street Cleaning and Litter' show a consistent spread of cases throughout the year, while others like 'Geneal' may exhibit more variability. The heatmap can be leveraged to anticipate service demand, optimize resource allocation, and enhance service delivery efficiency based on the observed patterns. Further analysis might explore the reasons behind high and low case volumes and investigate any anomalies to ensure responsive and effective service management.*

### Tree map

In [ ]:
# aggregate the data for the treemap
treemap_data = df.groupby(['rec_year', 'category', 'service_type']).size().reset_index(name = 'total_cases')

# rename column
treemap_data.rename(columns = {'rec_year': 'year'}, inplace = True)

In [ ]:
# create treemap
fig_treemap = px.treemap(treemap_data,
                         path = ['category', 'service_type'],   # define hierarchy
                         values = 'total_cases',   # size of the grid
                         title = 'Distribution of Service Types Across Categories Over Time')

fig_treemap.update_layout(
    margin = dict(t = 50, l = 25, r = 25, b = 25),
    title = {'text': "Distribution of Service Types Across Categories",
             'y': 0.94,
             'x': 0.5,
             'xanchor': 'center',
             'yanchor': 'top'
             }
    )

# show figure
fig_treemap.show()

*The treemap provides a revealing snapshot of service demand distribution across various categories. Notably, requests for parking infringement reviews dominate the 'General' category, standing out as the most high-demand service across the board. This significant volume suggests a potential area for policy review or enhanced enforcement measures, as it may reflect widespread parking violations.*

*Graffiti removal emerges as the second most sought-after service, highlighting the need for resource allocation to manage this issue effectively. The prominence of these two service types may necessitate a reassessment of current policies and initiatives to address the underlying causes of these frequent requests.*

*On the other end of the spectrum, 'Asset maintenance' appears to attract the fewest requests, indicating a lower relative demand or, possibly, higher efficiency in addressing these issues. In contrast, the 'Waste, Street Cleaning, and Litter' category, along with 'General' services, show a higher incidence of requests. This pattern points to waste management and general municipal services as significant areas of public concern. Such insights are invaluable for city planners and service coordinators, offering a data-driven basis for budgeting and staffing decisions to improve services that residents frequently utilize.*

### Stacked area chart

In [ ]:
# create a stacked area chart
fig_stackedarea = px.area(mth_sum_category, x = 'date', y = 'total_cases', color = 'category')

fig_stackedarea.update_layout(
    margin = dict(t = 50, l = 25, r = 25, b = 25),
    title = {'text': "Cumulative Count of Service Requests Over Time by Category",
             'y': 0.95,
             'x': 0.44,
             'xanchor': 'center',
             'yanchor': 'top'
             })

# rename y and x axis label
fig_stackedarea.update_layout(yaxis_title = 'Sum of Service Requests Across All Categories', xaxis_title = 'Date')

# show figure
fig_stackedarea.show()

*The stacked area chart suggests a data recording gap between late 2016 and 2018, as indicated by the service request count dropping to zero. This likely points to an issue in data collection rather than a complete lack of service requests. Following this period, there is a marked resurgence in service requests, with the overall count surpassing 5000 cases from March 2018 onwards. The 'General' category sees an especially sharp increase, accounting for over 2000 requests per month by mid-2018, which may reflect an increased need for these services or a change in how cases are reported. The demand for graffiti removal services appears stable, suggesting a consistent need for this service throughout the time frame. In contrast, the asset maintenance category shows a downward trend, possibly indicating enhanced efficiency in infrastructure upkeep or a shift in service reporting protocols.*

### Scatter plot

Find the relationship between visitor numbers and service requests in the 'Waste, Street Cleaning and Litter' category and visually inspect potential correlations between two quantitative variables using scatter plot.

In [ ]:
# filter the data
cat_waste = df[(df['category'] == 'Waste, Street Cleaning and Litter')]    # & (df['date_completed'].isna())

# group 'cat_waste' by year and month and count the number of cases
waste_cases_by_month = cat_waste.groupby(['rec_year', 'rec_month']).size().reset_index(name = 'waste_cases')

# merge 'sum_contacts' with 'waste_cases_by_month' on year and month
waste_cases_by_month = pd.merge(sum_contacts, waste_cases_by_month, left_on = ['year', 'month'], right_on = ['rec_year', 'rec_month'])

# drop redunant 'rec_year' and	'rec_month' columns
waste_cases_by_month.drop(['rec_year', 'rec_month'], axis = 1, inplace = True)

In [ ]:
waste_cases_by_month.head(3)

,year,month,number_of_contacts,date,waste_cases
0,2014,11,164042,2014-11,565
1,2014,12,193392,2014-12,735
2,2015,1,206044,2015-01,786


In [ ]:
# create scatter plot
fig_scatter = px.scatter(waste_cases_by_month, x = 'number_of_contacts', y = 'waste_cases',
                         labels = {'number_of_contacts': 'Number of Visitors',
                                   'waste_cases': 'Waste Service Requests'},
                         title = 'Relationship Between Visitor Numbers and Waste Service Requests')

fig_scatter.show()